
#augment dataset



In [ ]:
import os
import cv2
import numpy as np
import albumentations as A
from albumentations.pytorch import ToTensorV2
from tqdm import tqdm
from pathlib import Path
import shutil

# Set the paths
input_image_folder = 'mokolwane'  # Path to original images
output_image_folder = 'mokolwane_augmented'  # Path for augmented images

# Define Albumentations augmentations
transform = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.Rotate(limit=30, p=0.5),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), p=1.0),
    A.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1, p=0.5),
    A.GaussianBlur(blur_limit=(3, 7), p=0.3),
    ToTensorV2()
], bbox_params=A.BboxParams(format='yolo', label_fields=['category_ids']))

def denormalize(image, mean, std):
    mean = np.array(mean)
    std = np.array(std)
    image = image * std + mean
    image = np.clip(image, 0, 1)
    return image

# Iterate through images and apply augmentations
image_files = list(Path(input_image_folder).rglob('*.jpg'))

for image_path in tqdm(image_files, desc="Augmenting images"):
    if '/results/' in str(image_path):
        continue
    elif 'unlabeled_images' in str(image_path):
        continue
    elif 'unlabeled_images_original' in str(image_path):
        continue

    image = cv2.imread(str(image_path))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    annotation_file = image_path.with_suffix('.txt')
    bboxes, category_ids = [], []
    if annotation_file.exists():
        with open(str(annotation_file), 'r') as f:
            annotations = f.readlines()
        for annotation in annotations:
            parts = annotation.strip().split()
            category_ids.append(int(parts[0]))
            bboxes.append([float(x) for x in parts[1:]])

    augmented = transform(image=image, bboxes=bboxes, category_ids=category_ids)
    augmented_image = augmented['image']
    augmented_bboxes = augmented['bboxes']

    augmented_image = augmented_image.permute(1, 2, 0).numpy()
    augmented_image = denormalize(augmented_image, mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
    augmented_image = (augmented_image * 255).astype(np.uint8)

    relative_path = image_path.relative_to(input_image_folder)
    output_subdir = os.path.join(output_image_folder, relative_path.parent)
    os.makedirs(output_subdir, exist_ok=True)

    augmented_image_name = image_path.stem + "_augmented" + image_path.suffix
    augmented_image_path = os.path.join(output_subdir, augmented_image_name)
    cv2.imwrite(augmented_image_path, cv2.cvtColor(augmented_image, cv2.COLOR_RGB2BGR))

    augmented_annotation_name = image_path.stem + "_augmented.txt"
    augmented_annotation_path = os.path.join(output_subdir, augmented_annotation_name)
    with open(augmented_annotation_path, 'w') as f:
        for bbox, category_id in zip(augmented_bboxes, category_ids):
            f.write(f"{category_id} " + " ".join(map(str, bbox)) + "\n")

    original_image_output_path = os.path.join(output_subdir, image_path.name)
    shutil.copy(str(image_path), original_image_output_path)

    if annotation_file.exists():
        original_annotation_output_path = os.path.join(output_subdir, annotation_file.name)
        shutil.copy(str(annotation_file), original_annotation_output_path)

print("Augmentation complete!")


def copy_txt_files(source_dir, destination_dir):
  for filename in os.listdir(source_dir):
    if filename.endswith(".txt"):
      source_path = os.path.join(source_dir, filename)
      destination_path = os.path.join(destination_dir, filename)
      shutil.copy2(source_path, destination_path)

source_directory = 'mokolwane/val_mokolwane/images'
destination_directory = 'mokolwane/val_mokolwane/labels'

copy_txt_files(source_directory, destination_directory)